This is official pytorch tutorial: <a href=https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py> Blitz Tutorial<a>

What is done in this tutorial:
    1. Load and normalize the CIFAR10 training and test datasets using torchvision
    2. Define a Convolutional Neural Network
    3. Define a loss function
    4. Train the network on the training data
    5. Test the network on the test data

TORCHVISION:
    The torchvision package consists of popular datasets, model architectures,
    and common image transformations for computer vision.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.functional import one_hot

import matplotlib.pyplot as plt
import numpy as np
import h5py
import pickle
from itertools import chain
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
embeddings = h5py.File("../../data/embeddings.h5", "r")
with open("../../data/seq_anno_hash.pickle", 'rb') as handle:
    proteins_and_hashes = pickle.load(handle)

In [3]:
label_prot = [list(map(lambda x:[x,key], list(proteins_and_hashes[key].keys()))) for key in proteins_and_hashes]
label_prot = list(chain.from_iterable(label_prot))
label_prot = pd.DataFrame(label_prot, columns=["prot_id", "label"])

X_train, X_test = train_test_split(label_prot, test_size = 0.1, train_size=0.9, random_state=42, stratify=label_prot["label"])
print(len(X_train))
print(len(X_test))
X_train

4638
516


,prot_id,label
1240,P31716,G_SP
2049,Q24537,G
1660,Q9VWW0,G
3495,Q0GGX2,G
1260,P13423,G_SP
...,...,...
1801,P18146,G
4276,Q21874,SP_TM
2501,Q13415,G
2687,Q9HGP0,G


In [4]:
train_embeddings = list()
train_labels = list()
train_protein_ids = list()
for index, row in X_train.iterrows():
    hash_code = proteins_and_hashes[row["label"]][row["prot_id"]][2]
    mean_embedding = np.mean(embeddings.get(hash_code), axis=0)
    label = row["label"]
    id = row["prot_id"]
    train_protein_ids.append(id)
    train_embeddings.append(mean_embedding)
    train_labels.append(label)

test_embeddings = list()
test_labels = list()
test_protein_ids = list()
for index, row in X_test.iterrows():
    hash_code = proteins_and_hashes[row["label"]][row["prot_id"]][2]
    mean_embedding = np.mean(embeddings.get(hash_code), axis=0)
    label = row["label"]
    id = row["prot_id"]
    test_protein_ids.append(id)
    test_embeddings.append(mean_embedding)
    test_labels.append(label)

### Create dataloader

In [5]:
label_mappings = {
    'G_SP': 0,
     'G': 1,
     'SP_TM': 2,
     'TM': 3
}
reverse_label_mappings = {val: key for key, val in label_mappings.items()}

In [8]:
train_embeddings = torch.Tensor(np.array(train_embeddings))
train_labels = torch.Tensor([label_mappings[label] for label in  train_labels])

In [9]:
test_embeddings = torch.Tensor(np.array(test_embeddings))
test_labels = torch.Tensor([label_mappings[label] for label in  test_labels])

In [10]:
train_labels = one_hot(train_labels.to(torch.int64), 4)
dataset = TensorDataset(train_embeddings, train_labels)
dataloader = DataLoader(dataset, batch_size=4)

In [11]:
test_labels = one_hot(test_labels.to(torch.int64), 4)
test_dataset = TensorDataset(test_embeddings, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [72]:
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    """ Define a model with two convolution layers each followed by a Max Pooling layer
    Then the embeddings are Flattened and forwarded into two FCls.
    The last layer is a FCL with 10 neurons.
    """
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.conv1 = nn.Conv1d(512, 512, kernel_size=3, padding=1, stride=1)

        self.fc2 = nn.Linear(512, 512)
        self.conv1 = nn.Conv1d(512, 512, kernel_size=3, padding=1, stride=1)

        self.fc3 = nn.Linear(512, 256)
        self.conv3 = nn.Conv1d(256, 256, kernel_size=3, padding=1, stride=1)

        self.fc4 = nn.Linear(256, 4)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.conv1(x))

        x = F.relu(self.fc2(x))
        #x = F.relu(self.conv2(x))

        x = F.relu(self.fc3(x))
        #x = F.relu(self.conv3(x))

        x = self.fc4(x)
        #x = F.softmax(x, dim=4)
        return x


net = CNN()

In [73]:
import torch.optim as optim

# The function is indicating the performance of the model.
# During the training process this function should be minimized
criterion = nn.CrossEntropyLoss()

# The minimization is achieved through Stochastic Gradient Descent
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [30]:
input = torch.randn(2, 10)
m = nn.Conv1d(10, 7, 1, stride=2)
m(input)

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [7, 10, 1], but got 2-dimensional input of size [2, 10] instead

In [74]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        """
        shape of inputs: torch.Size([4, 3, 32, 32])
            Batchsize: 4
            Channels: 3 (Red, Green, Blue)
            Image size: 32 x 32

        labels: tensor([9, 3, 0, 3])
            9: class of image 0 in batch
            3: class of image 1 in batch
            ...
        """
        inputs, labels = data

        """ zero the parameter gradients after every batch
        This is necessary because the gradients (directions of how the weigths and biases
        will be updated) are accumulated in each backward pass.
        https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        """
        optimizer.zero_grad()  # SGD

        # forward + backward + optimize
        # shape outputs: torch.Size([4, 10])
        # for every image a prediction
        #print(f"{inputs}")
        outputs = net(inputs)
        #print(f"{outputs} \t {labels}")

        # the first iteration CrossEntropy: tensor(2.3100, grad_fn=<NllLossBackward0>)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # running loss after 3 iterations: 6.894119024276733
        # Why is the loss added?
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {(running_loss / 99):.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 100] loss: 1.178
[1, 200] loss: 1.043
[1, 300] loss: 1.045
[1, 400] loss: 0.858
[1, 500] loss: 0.555
[1, 600] loss: 0.414
[1, 700] loss: 0.352
[1, 800] loss: 0.453
[1, 900] loss: 0.492
[1, 1000] loss: 0.387
[1, 1100] loss: 0.302
[2, 100] loss: 0.316
[2, 200] loss: 0.315
[2, 300] loss: 0.221
[2, 400] loss: 0.264
[2, 500] loss: 0.189
[2, 600] loss: 0.109
[2, 700] loss: 0.118
[2, 800] loss: 0.168
[2, 900] loss: 0.209
[2, 1000] loss: 0.198
[2, 1100] loss: 0.104
[3, 100] loss: 0.146
[3, 200] loss: 0.093
[3, 300] loss: 0.113
[3, 400] loss: 0.143
[3, 500] loss: 0.103
[3, 600] loss: 0.079
[3, 700] loss: 0.098
[3, 800] loss: 0.104
[3, 900] loss: 0.130
[3, 1000] loss: 0.149
[3, 1100] loss: 0.070
[4, 100] loss: 0.126
[4, 200] loss: 0.067
[4, 300] loss: 0.077
[4, 400] loss: 0.093
[4, 500] loss: 0.096
[4, 600] loss: 0.054
[4, 700] loss: 0.106
[4, 800] loss: 0.075
[4, 900] loss: 0.093
[4, 1000] loss: 0.120
[4, 1100] loss: 0.045
Finished Training


### Evaluate the model on the test data
This could be done with TorchMetrics but we will do this manually here

In [69]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)

        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(test_labels)} test embeddings: {(100 * correct / total):.2f}')

Accuracy of the network on the 516 test embeddings: 95.74


In [70]:
# prepare to count predictions for each class
classes = list(label_mappings.keys())
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        _, labels = torch.max(labels, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                         accuracy))

"""
Accuracy for class G_SP  is: 0.0 %
Accuracy for class G     is: 100.0 %
Accuracy for class SP_TM is: 0.0 %
Accuracy for class TM    is: 0.0 %
"""

Accuracy for class G_SP  is: 96.2 %
Accuracy for class G     is: 99.3 %
Accuracy for class SP_TM is: 81.0 %
Accuracy for class TM    is: 89.7 %


'\nAccuracy for class G_SP  is: 0.0 %\nAccuracy for class G     is: 100.0 %\nAccuracy for class SP_TM is: 0.0 %\nAccuracy for class TM    is: 0.0 %\n'

Create hashsum